# Example 3:Ridge Regression
This example try to show how to use our PLQ Composite Decomposition tool to perform Ridge Regression.

# 1. Data Generation

In [1]:
from plqcom.PLQLoss import PLQLoss
from plqcom.PLQProperty import plq_to_rehloss
from plqcom.ReHProperty import affine_transformation
import numpy as np
from rehline import ReHLine

n_samples, n_features = 1000, 5
rng = np.random.RandomState(0)
X = rng.randn(n_samples, n_features)
beta = rng.randn(n_features)
y = np.dot(X, beta) + rng.normal(scale=0.1, size=n_samples)

# 2.Create and Decompose the PLQ Loss

In [2]:
plqloss = PLQLoss(quad_coef={'a': np.array([1.]), 'b': np.array([0.]), 'c': np.array([0.])},
                  cutpoints=np.array([]))

In [3]:
rehloss = plq_to_rehloss(plqloss)

In [4]:
rehloss.rehu_cut, rehloss.rehu_coef, rehloss.rehu_intercept

(array([[inf],
        [inf]]),
 array([[ 1.41421356],
        [-1.41421356]]),
 array([[ 0.],
        [-0.]]))

# 3. Broadcast to all samples

In [5]:
rehloss = affine_transformation(rehloss, n=X.shape[0], c=1, p=-1, q=y)

# 4. Use the ReHLine to solve the problem

In [6]:
clf = ReHLine(loss={'name': 'custom'}, C=1)
clf.Tau, clf.S, clf.T = rehloss.rehu_cut, rehloss.rehu_coef, rehloss.rehu_intercept
clf.fit(X=X)
print('sol privided by rehline: %s' % clf.coef_)

sol privided by rehline: [ 0.31026702 -0.7385321  -1.53889185 -0.56081954 -1.60477043]


Compare with the solution provided by sklearn

In [7]:
from sklearn.linear_model import Ridge
clf1 = Ridge(alpha=0.5)
clf1.fit(X, y)
print('sol privided by sklearn: %s' % clf1.coef_)

sol privided by sklearn: [ 0.31017419 -0.73849146 -1.53893293 -0.56084128 -1.60476756]
